In [1]:
import pandas as pd
import re
import requests
import praw    #to access reddit data
from praw.models import MoreComments

In [2]:
requests.get('https://www.reddit.com/r/wallstreetbets/new/', headers = {'User-agent': 'your bot 0.1'})

<Response [200]>

In [3]:
reddit = praw.Reddit(
  client_id = "Your client_id",
  client_secret = "Your client_secret",
  user_agent = "Comment Extraction (by u/user name)"
)

In [4]:
subreddit = reddit.subreddit('wallstreetbets')

In [5]:
ticker_df = pd.read_csv('nasdaq_screener.csv').rename(columns = {'Symbol': 'Term', 'Name': 'Company_Name'})
#ticker_df

In [6]:
ticker_list = ticker_df['Term'].tolist()
ticker_list_lite = ['SPY', 'GME', 'AMC', 'SPCE', 'BABA', 'AMD', 'AMZN', 'SOFI', 'TSLA', 'WKHS', 
                    'CLOV', 'CLNE', 'JD', 'IQ', 'BB', 'WISH', 'WEN', 'NIO', 'CRSR', 'NLY',
                    'FOMO', 'QQQ', 'AAPL', 'TTCF', 'FUBO', 'NVDA', 'CLF']

Method 1

In [7]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
all_tickers = {}
hot_tickers = {}

for i in range (0, len(ticker_list)):
    ticker = ticker_list[i]
    all_tickers[ticker] = 1
# all_tickers

In [9]:
regexPattern = r'\b([A-Z]+)\b'

sentimental_score = {}
final_1 = {}

In [10]:
# nltk.download('vader_lexicon')

In [11]:
vader = SentimentIntensityAnalyzer()

In [12]:
for submission in subreddit.top('day'):
    strings = [submission.title]
    submission.comments.replace_more(limit = 0)
    for comment in submission.comments.list():
        strings.append(comment.body)
#        break
    for s in strings:
        for phrase in re.findall(regexPattern, s):
            if phrase in all_tickers:
                score1 = vader.polarity_scores(s)
#                 print(score1)
                if phrase not in sentimental_score:
                    sentimental_score[phrase] = score1
                else:
                    for key,_ in score1.items():
                        sentimental_score[phrase][key] += score1[key]
#         break
                if phrase not in hot_tickers:
                    hot_tickers[phrase] = 1
                else:
                    hot_tickers[phrase] += 1
        tickers1 = list(sentimental_score)
        for ticker1 in tickers1:
            final_1[ticker1] = sentimental_score[ticker1]['compound']
#     break

In [13]:
# sentimental_score
final_1

{'DD': 10.1228,
 'F': 1.7602000000000002,
 'U': -0.5876999999999999,
 'EXP': 0.0,
 'SENS': 0.0,
 'D': 6.6932,
 'PROS': -0.4466,
 'YOU': -2.1176,
 'ON': -0.4466,
 'FOR': -0.4466,
 'NEXT': -0.4466,
 'ARE': -1.3096999999999999,
 'JOBS': -0.4466,
 'SAM': -0.5423,
 'IRS': -1.3133,
 'RH': -0.9683000000000003,
 'LMAO': 0.8789,
 'PLTR': 4.9261,
 'A': 8.439300000000001,
 'IT': -0.033100000000000004,
 'ATH': 0.5234,
 'GME': 9.632199999999997,
 'AMD': 10.174900000000001,
 'C': 3.0509,
 'RIDE': 0.1487,
 'MCD': 0.4939,
 'IS': 0.0,
 'HOG': 1.3135000000000001,
 'DNUT': 0.34,
 'T': -0.7248000000000001,
 'BOOM': 0.0,
 'SNDL': 2.5476,
 'MAT': 0.0,
 'AMC': 10.714699999999999,
 'SOFI': -0.7959,
 'W': 0.0,
 'O': 1.1925,
 'HAS': 0.0,
 'SO': 0.0,
 'BB': 1.8048999999999995,
 'BNR': 0.0,
 'PULM': 0.2263,
 'Z': 0.0,
 'AAPL': 0.0,
 'B': 4.445500000000001,
 'TX': 0.0,
 'MSFT': 0.1179,
 'OG': -0.22349999999999998,
 'MU': 2.1197,
 'TSLA': 0.8846999999999999,
 'SPCE': 4.4815,
 'NVDA': 1.975,
 'BYND': 0.0434,
 'GO': 

In [14]:
series = pd.Series(hot_tickers).sort_values(ascending = False)
series[:10]

GME     98
AMC     90
DD      79
AMD     63
CLOV    58
A       49
WISH    29
BB      27
CLF     26
RH      21
dtype: int64

In [15]:
final_df = pd.DataFrame(pd.Series(final_1)).loc[series[:10].index]
final_df

,0
GME,9.6322
AMC,10.7147
DD,10.1228
AMD,10.1749
CLOV,11.9255
A,8.4393
WISH,11.0521
BB,1.8049
CLF,6.8131
RH,-0.9683


In [16]:
final_df['weights'] = final_df[0] / sum(final_df[0]) *100

In [17]:
final_df

,0,weights
GME,9.6322,12.083873
AMC,10.7147,13.441900
DD,10.1228,12.699345
AMD,10.1749,12.764706
CLOV,11.9255,14.960884
A,8.4393,10.587345
WISH,11.0521,13.865178
BB,1.8049,2.264299
CLF,6.8131,8.547231
RH,-0.9683,-1.214760


Method 2

In [18]:
data_input = subreddit.top('day', limit = 1000)
sum = [0] * len(ticker_list_lite) # our output array
counttotal = 0 # total number of comment read
submissions_counter = 0

In [19]:
for submissions in data_input:
    
  if not submissions.stickied:
    
    submissions_counter += 1
    
    if submissions_counter > 5:
        
      comments = submissions.comments
    
      for comment in comments:
            
        if isinstance(comment, MoreComments):
            continue
            
        counttotal += 1
        
        for i, ticker in enumerate(ticker_list_lite):
            
          if ticker in comment.body:
            
            sum[i] += 1

In [20]:
output = pd.DataFrame(data = {'Tick': ticker_list_lite, 'Counts': sum}).sort_values(['Counts'], ascending = False)
print('Total comments read: ', counttotal)
print(output[output['Counts'] > 0][:10])

Total comments read:  874
    Tick  Counts
1    GME      15
2    AMC      13
5    AMD      11
26   CLF      10
21   QQQ       7
15  WISH       5
14    BB       4
0    SPY       4
10  CLOV       3
3   SPCE       3
